In [1]:
import numpy as np
import pandas as pd

from sklearn.utils.class_weight import compute_class_weight

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import Pool, CatBoostClassifier

from sklearn.metrics import precision_score

c:\Users\Mat\miniconda3\envs\ml\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
full_train = pd.read_csv('datasets/cleaned_original+dollar+bitcoin+ratings_train_v3.csv', delimiter=',')
full_test = pd.read_csv('datasets/cleaned_original+dollar+bitcoin+ratings_test_v3.csv', delimiter=',')

In [3]:
# full_dataset
x_train = full_train.drop('label', axis=1)
y_train = full_train['label']

x_val = full_test.drop('label', axis=1)
y_val = full_test['label']

In [4]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

# scale_pos_weight = int(class_weights[1] / class_weights[0])
scale_pos_weight = y_train.shape[0]/ y_train.sum()
# scale_pos_weight= 50
class_weights, scale_pos_weight

({0.0: 0.512373237867053, 1.0: 20.70489726991275}, 41.4097945398255)

In [6]:
catbcl = CatBoostClassifier(iterations = 400, depth = 12, learning_rate = 0.03, loss_function = "Logloss", class_weights=class_weights,
                            task_type = "GPU", verbose = True)

In [20]:
gbcl = XGBClassifier(booster = "gbtree", max_depth = 0, verbosity = 0, scale_pos_weight=scale_pos_weight)

forest_19_5 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                                                                    class_weight=class_weights)

forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                        min_samples_leaf=5, class_weight=class_weights, min_samples_split=10,
                                        random_state=404)

In [10]:
y_pred = catbcl.predict(x_val)
precision_score(y_val, y_pred)

0.053683996475293164

In [8]:
def catbcl_train_func(models, x_train, y_train, x_val, y_val):
    for model in models:
        name = model.__class__.__name__
        print(name + ": ")
        
        model.fit(x_train, y_train, eval_set=(x_val, y_val))
        # print("Обучение завершено!")
        y_pred = model.predict(x_val)

        # f1 = f1_score(Y_test, Y_pred)
        precision = precision_score(y_val, y_pred)

        # print("-" * 10)
        # print(f1)
        print(precision)

        
catbcl_train_func([catbcl], x_train, y_train, x_val, y_val)# svc_poly, svc_2nd_verse, gauss_nb, qda

CatBoostClassifier: 
0:	learn: 0.6759919	test: 0.6834569	best: 0.6834569 (0)	total: 154ms	remaining: 1m 1s
1:	learn: 0.6597093	test: 0.6760796	best: 0.6760796 (1)	total: 672ms	remaining: 2m 13s
2:	learn: 0.6445625	test: 0.6681220	best: 0.6681220 (2)	total: 1.16s	remaining: 2m 33s
3:	learn: 0.6301212	test: 0.6625970	best: 0.6625970 (3)	total: 1.46s	remaining: 2m 24s
4:	learn: 0.6168421	test: 0.6571149	best: 0.6571149 (4)	total: 2.71s	remaining: 3m 33s
5:	learn: 0.6043192	test: 0.6512910	best: 0.6512910 (5)	total: 3.61s	remaining: 3m 56s
6:	learn: 0.5925819	test: 0.6460568	best: 0.6460568 (6)	total: 4.11s	remaining: 3m 50s
7:	learn: 0.5814946	test: 0.6408153	best: 0.6408153 (7)	total: 4.42s	remaining: 3m 36s
8:	learn: 0.5714846	test: 0.6378525	best: 0.6378525 (8)	total: 5.1s	remaining: 3m 41s
9:	learn: 0.5625240	test: 0.6354619	best: 0.6354619 (9)	total: 5.49s	remaining: 3m 34s
10:	learn: 0.5534696	test: 0.6316242	best: 0.6316242 (10)	total: 6.05s	remaining: 3m 33s
11:	learn: 0.5449409	t

In [21]:
models = [gbcl, forest_19_5, forest_19_5_v2]

In [22]:
for model in models:
    name = model.__class__.__name__
    print(name + ": ")
    
    model.fit(x_train, y_train)
    # print("Обучение завершено!")

    y_pred = model.predict(x_val)
    precision = precision_score(y_val, y_pred)
    
    print(precision)
    print("-" * 10)

XGBClassifier: 


c:\Users\Mat\miniconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Mat\miniconda3\envs\ml\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.021064896639304557
----------
RandomForestClassifier: 
0.11163522012578617
----------
RandomForestClassifier: 
0.13016759776536313
----------


Биткоины, информация о городах +0.1

---

In [23]:
full_train.columns

Index(['label', 'hex_lat', 'hex_lon', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7',
       'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17',
       'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27',
       'f28', 'f29', 'f30', 'dollar', 'bitcoin', 'subject_score', 'city_score',
       'population'],
      dtype='object')

Оставим f-ки

In [27]:
train_data_1 = full_train.drop(['hex_lat', 'hex_lon', 'dollar', 'bitcoin', 'subject_score', 'city_score',
       'population'], axis=1)
test_data_1 = full_test.drop(['hex_lat', 'hex_lon', 'dollar', 'bitcoin', 'subject_score', 'city_score',
       'population'], axis=1)

x_train_1 = train_data_1.drop('label', axis=1)
y_train_1 = train_data_1['label']

x_val_1 = test_data_1.drop('label', axis=1)
y_val_1 = test_data_1['label']

In [28]:
train_1_forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                         min_samples_leaf=5, class_weight=class_weights, min_samples_split=10, random_state=404)

train_1_forest_19_5_v2.fit(x_train_1, y_train_1)

train_1_y_pred = train_1_forest_19_5_v2.predict(x_val_1)
precision_score(y_val_1, train_1_y_pred)

0.10738255033557047

Мы молодцы

Оставим 'subject_score', 'city_score', 'population' и f-ки

In [31]:
train_data_2 = full_train.drop(['hex_lat', 'hex_lon', 'dollar', 'bitcoin'], axis=1)
test_data_2 = full_test.drop(['hex_lat', 'hex_lon', 'dollar', 'bitcoin'], axis=1)

x_train_2 = train_data_2.drop('label', axis=1)
y_train_2 = train_data_2['label']

x_val_2 = test_data_2.drop('label', axis=1)
y_val_2 = test_data_2['label']

In [32]:
train_2_forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                         min_samples_leaf=5, class_weight=class_weights, min_samples_split=10, random_state=404)

train_2_forest_19_5_v2.fit(x_train_2, y_train_2)

train_2_y_pred = train_2_forest_19_5_v2.predict(x_val_2)
precision_score(y_val_2, train_2_y_pred)

0.10830527497194165

От доллара и биткоина мало что зависит

---

Оставим 'dollar', 'bitcoin' и f-ки

In [33]:
train_data_3 = full_train.drop(['hex_lat', 'hex_lon', 'subject_score', 'city_score', 'population'], axis=1)
test_data_3 = full_test.drop(['hex_lat', 'hex_lon', 'subject_score', 'city_score', 'population'], axis=1)

x_train_3 = train_data_3.drop('label', axis=1)
y_train_3 = train_data_3['label']

x_val_3 = test_data_3.drop('label', axis=1)
y_val_3 = test_data_3['label']

In [34]:
train_3_forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                         min_samples_leaf=5, class_weight=class_weights, min_samples_split=10, random_state=404)

train_3_forest_19_5_v2.fit(x_train_3, y_train_3)

train_3_y_pred = train_3_forest_19_5_v2.predict(x_val_3)
precision_score(y_val_3, train_3_y_pred)

0.11472742066720912

---

Отставим 'hex_lat', 'hex_lon' )))

In [35]:
train_data_4 = full_train.drop(['dollar', 'bitcoin', 'subject_score', 'city_score',
       'population'], axis=1)
test_data_4 = full_test.drop(['dollar', 'bitcoin', 'subject_score', 'city_score',
       'population'], axis=1)

x_train_4 = train_data_4.drop('label', axis=1)
y_train_4 = train_data_4['label']

x_val_4 = test_data_4.drop('label', axis=1)
y_val_4 = test_data_4['label']

In [36]:
train_4_forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                         min_samples_leaf=5, class_weight=class_weights, min_samples_split=10, random_state=404)

train_4_forest_19_5_v2.fit(x_train_4, y_train_4)

train_4_y_pred = train_4_forest_19_5_v2.predict(x_val_4)
precision_score(y_val_4, train_4_y_pred)

0.11705831157528286

---

Минус биткоин и доллар

In [40]:
train_data_5 = full_train.drop(['dollar', 'bitcoin'], axis=1)
test_data_5 = full_test.drop(['dollar', 'bitcoin'], axis=1)

x_train_5 = train_data_5.drop('label', axis=1)
y_train_5 = train_data_5['label']

x_val_5 = test_data_5.drop('label', axis=1)
y_val_5 = test_data_5['label']

In [41]:
train_5_forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                         min_samples_leaf=5, class_weight=class_weights, min_samples_split=10, random_state=404)

train_5_forest_19_5_v2.fit(x_train_5, y_train_5)

train_5_y_pred = train_5_forest_19_5_v2.predict(x_val_5)
precision_score(y_val_5, train_5_y_pred)

0.11536748329621381

---

Уберем 'subject_score', 'city_score', 'population'

In [42]:
train_data_6 = full_train.drop(['subject_score', 'city_score', 'population'], axis=1)
test_data_6 = full_test.drop(['subject_score', 'city_score', 'population'], axis=1)

x_train_6 = train_data_6.drop('label', axis=1)
y_train_6 = train_data_6['label']

x_val_6 = test_data_6.drop('label', axis=1)
y_val_6 = test_data_6['label']

In [43]:
train_6_forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                         min_samples_leaf=5, class_weight=class_weights, min_samples_split=10, random_state=404)

train_6_forest_19_5_v2.fit(x_train_6, y_train_6)

train_6_y_pred = train_6_forest_19_5_v2.predict(x_val_6)
precision_score(y_val_6, train_6_y_pred)

0.12640283520378026

---

Уберем subject_score

In [45]:
train_data_7 = full_train.drop(['subject_score'], axis=1)
test_data_7 = full_test.drop(['subject_score'], axis=1)

x_train_7 = train_data_7.drop('label', axis=1)
y_train_7 = train_data_7['label']

x_val_7 = test_data_7.drop('label', axis=1)
y_val_7 = test_data_7['label']

In [46]:
train_7_forest_19_5_v2 = RandomForestClassifier(max_depth = 19, n_estimators = 500, criterion = "gini", n_jobs = 5,
                                         min_samples_leaf=5, class_weight=class_weights, min_samples_split=10, random_state=404)

train_7_forest_19_5_v2.fit(x_train_7, y_train_7)

train_7_y_pred = train_7_forest_19_5_v2.predict(x_val_7)
precision_score(y_val_7, train_7_y_pred)

0.1329288472145615

### Лучший результат выходит только при использовании всех наших колонок.